In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'mps'
print(device)

mps
mps


# data transform --> norm, type casting, crop, rotation

In [3]:
train_transform = transforms.Compose([
    transforms.ToTensor(), # 0~1 numpy -> tensor,
    # 0 ~ 1 0-0.5/0.5:-1 (1-0.5)/0.5, -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # 좌우 반전
    transforms.RandomHorizontalFlip(p=0.5), 
])

test_transform = transforms.Compose([
    transforms.ToTensor(), # 0~1 numpy -> tensor,
    # 0 ~ 1 0-0.5/0.5:-1 (1-0.5)/0.5, -1~1
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [4]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainset.data.shape, len(trainset.targets), trainset.targets[:10]

((50000, 32, 32, 3), 50000, [6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

((50000, 32, 32, 3), 50000, [6, 9, 9, 4, 1, 1, 2, 7, 8, 3])

In [6]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [8]:
images, labels = next(iter(trainloader))

In [9]:
images.shape, labels.shape, images.min(), images.max()

(torch.Size([100, 3, 32, 32]), torch.Size([100]), tensor(-1.), tensor(1.))

(torch.Size([100, 3, 32, 32]), torch.Size([100]), tensor(-1.), tensor(1.))

In [10]:
import matplotlib.pyplot as plt

In [ ]:
images[0].permute(1,2,0).shape

In [ ]:
img = images[1].permute(1,2,0)

In [ ]:
img = img * 0.5 + 0.5

In [ ]:
img.min(), img.max()

In [ ]:
plt.title(classes[labels[1].item()])
plt.imshow(img)
plt.show()

# model init 

In [11]:
class cifar_cnn(nn.Module):
    def __init__(self):
        super(cifar_cnn, self).__init__()
        # layer
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=4, kernel_size=(3,3)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(3,3)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # 32 x 5 x 5
        
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(0.5), 
            nn.Linear(32*5*5, 64),
            nn.ReLU(inplace=True),
            
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            
            nn.Linear(32, 10)

        )
        
    
    def forward(self, x):
        # 순전파
        
        out = self.conv_layer(x)
        # bx16x5x5 --> 1d --> 400
        out = out.view(out.shape[0], -1)
        
        out = self.fc_layer(out)
        
        
        return out
        
    

In [12]:
model = cifar_cnn()

In [13]:
from torchsummary import summary

In [14]:
summary(model, input_size=((3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 30, 30]             112
              ReLU-2            [-1, 4, 30, 30]               0
            Conv2d-3           [-1, 16, 28, 28]             592
              ReLU-4           [-1, 16, 28, 28]               0
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 12, 12]           4,640
              ReLU-7           [-1, 32, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           9,248
              ReLU-9           [-1, 32, 10, 10]               0
        MaxPool2d-10             [-1, 32, 5, 5]               0
          Dropout-11                  [-1, 800]               0
           Linear-12                   [-1, 64]          51,264
             ReLU-13                   [-1, 64]               0
           Linear-14                   

# model learning 

In [15]:
learning_rate = 0.01 #1e-2

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 10
model = model.to(device)
for _epoch in range(epochs):
    for it_batch, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        
        if (it_batch+1) % 30 == 0:
            print(f'epoch:{_epoch}, batch:{it_batch}, loss{loss.item()}')
            
            with torch.no_grad():
                model.eval()

                correct_ = 0
                total_ = 0

                for it_batch, (images, labels) in enumerate(testloader):
                    images = images.to(device)
                    labels = labels.to(device)                    
                    
                    outputs = model(images)

                    predcits = torch.argmax(outputs, axis=1)
                    correct_ += (labels == predcits).sum()
                    total_ += predcits.shape[0]

                acc = correct_/total_ * 100
                print(f'acc{acc:.2f}, correct{correct_}, total{total_}')
                
                model.train()


epoch:0, batch:29, loss2.1021535396575928
epoch:0, batch:29, loss2.1021535396575928
acc20.41, correct2041, total10000
acc20.41, correct2041, total10000
epoch:0, batch:59, loss2.0212440490722656
epoch:0, batch:59, loss2.0212440490722656
acc25.76, correct2576, total10000
acc25.76, correct2576, total10000
epoch:0, batch:89, loss1.864848017692566
epoch:0, batch:89, loss1.864848017692566
acc27.33, correct2733, total10000
acc27.33, correct2733, total10000
epoch:0, batch:119, loss1.9393582344055176
epoch:0, batch:119, loss1.9393582344055176
acc29.50, correct2950, total10000
acc29.50, correct2950, total10000
epoch:0, batch:149, loss1.8619422912597656
epoch:0, batch:149, loss1.8619422912597656
acc31.08, correct3108, total10000
acc31.08, correct3108, total10000
epoch:0, batch:179, loss2.011519193649292
epoch:0, batch:179, loss2.011519193649292
acc30.95, correct3095, total10000
acc30.95, correct3095, total10000
epoch:0, batch:209, loss1.8934205770492554
epoch:0, batch:209, loss1.8934205770492554


# evaluate  

In [ ]:
model = model.to('cpu')

In [ ]:

with torch.no_grad():
    model.eval()
    
    correct_ = 0
    total_ = 0

    for it_batch, (images, labels) in enumerate(testloader):
        
        outputs = model(images)
        
        predcits = torch.argmax(outputs, axis=1)
        correct_ += (labels == predcits).sum()
        total_ += predcits.shape[0]
    
    acc = correct_/total_ * 100
    print(f'acc{acc:.2f}, correct{correct_}, total{total_}')
    